In [31]:
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta
import requests as rq
from io import BytesIO
import re
import numpy as np
import statsmodels.api as sm

In [32]:
df = pd.read_excel('../stocks.xlsx', engine='openpyxl')
tickers = df['종목코드']

new_df = df[['종목코드','종목명']]
# new_df.set_index('종목코드', inplace=True) # 인덱스 설정

# 1년 기간동안의 종가 데이터 받기
fr = (date.today() + relativedelta(years=-1)).strftime('%Y%m%d') # 1년간 기간
to = (date.today()).strftime('%Y%m%d') # 오늘 날짜

In [33]:
code_lists = []
return_values =[]
res_lists=[]
for ticker in tickers:
    real_ticker=ticker
    ticker = str(ticker).zfill(6)
    url = f'''https://fchart.stock.naver.com/siseJson.nhn?symbol={ticker}&requestType=1&startTime={fr}&endTime={to}&timeframe=day'''
    data = rq.get(url).content
    data_price = pd.read_csv(BytesIO(data), encoding='utf-8')

    # 데이터 전처리
    price = data_price.iloc[:, 0:6]
    price.columns = ['날짜','시가','고가','저가','종가','거래량']
    price = price.dropna()
    price['날짜'] = price['날짜'].str.extract('(\d+)')
    price['날짜'] = pd.to_datetime(price['날짜'])
    price = pd.DataFrame(price)
    
    # 1년간 누적 수익률 구하기
    return_value = (price['종가'].iloc[-1] / price['종가'].iloc[0]) - 1
#     print(ticker, return_value)
    code_lists.append(real_ticker)
    return_values.append(return_value)
    
    # 일간 수익률 구하기
    try:
        ret = price['종가'].pct_change()[1:]
        ret_cum = np.log(1 + ret).cumsum()

        x = np.array(range(len(ret)))
        y = ret_cum.iloc[:].values

        reg = sm.OLS(y,x).fit()
        res = float((reg.params / reg.bse))
    except:
        res = np.nan
    res_lists.append(res)
    
temp_df = pd.DataFrame({'종목코드':code_lists, 'return':return_values, 'res':res_lists})

In [44]:
data_bind = new_df.merge(temp_df, how='left', on='종목코드')

In [45]:
data_bind

,종목코드,종목명,return,res
0,95570,AJ네트웍스,-0.163194,6.953520
1,54620,APS홀딩스,0.038174,1.783514
2,265520,AP시스템,-0.145228,-26.589196
3,211270,AP위성,0.590604,-9.813475
4,27410,BGF,-0.201905,-23.866876
...,...,...,...,...
1025,84110,휴온스글로벌,-0.349917,-42.256162
1026,24060,흥구석유,-0.371986,-54.275345
1027,10240,흥국,-0.113669,-12.535108
1028,189980,흥국에프엔비,-0.316520,-37.032692


In [48]:
data_bind.to_excel('yearmomentum.xlsx')

In [42]:
data_bind

,종목코드,종목명,return,res
0,95570,AJ네트웍스,-0.163194,6.953520
1,54620,APS홀딩스,0.038174,1.783514
2,265520,AP시스템,-0.145228,-26.589196
3,211270,AP위성,0.590604,-9.813475
4,27410,BGF,-0.201905,-23.866876
...,...,...,...,...
1025,84110,휴온스글로벌,-0.349917,-42.256162
1026,24060,흥구석유,-0.371986,-54.275345
1027,10240,흥국,-0.113669,-12.535108
1028,189980,흥국에프엔비,-0.316520,-37.032692
